In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.model_selection import train_test_split

In [66]:
timeseries_test = pd.read_csv('../Data/Output/timeseries_merged.csv',encoding='latin1')
timeseries_test.set_index('Date', inplace=True)
timeseries_test['difficulty'] = timeseries_test['difficulty'].apply(lambda x: '{:.2f}'.format(x))

In [3]:
timeseries_test.head()

,close,avg_block_size,blocks_size,cost_per_transaction_percent,cost_per_transaction,difficulty,est_transaction_volume_USD,est_transaction_volume,hash_rate,median_confirmation_time,...,utxo_count_stddev,utxo_count_min,utxo_count_max,news_stories_total,news_stories_positive,news_stories_negative,news_stories_neutral,Negative_Tweets,Neutral_Tweets,Positive_Tweets
Date,,,,,,,,,,,,,,,,,,,,,
4/24/2016,458.55,0.728040,66022.67805,1.385967,8.360698,178678000000.00,110486293.2,240501.2912,1172443.384,8.783333,...,27228.75866,36724621,36825214,6,2,0,4,7.868904,12.110776,18.395958
4/25/2016,461.43,0.874485,66133.73765,0.888460,6.861613,178678000000.00,168025613.1,362233.4608,1128032.650,9.683333,...,17998.12914,36825828,36893715,25,7,8,10,7.206912,36.133294,38.252961
4/26/2016,466.09,0.813180,66256.52779,1.129962,7.675906,178678000000.00,170332016.5,363972.8546,1341204.175,7.750000,...,16768.83707,36895435,36980350,32,14,14,4,7.127595,19.316951,19.296031
4/27/2016,444.69,0.875343,66369.44708,0.873767,6.111057,178678000000.00,167458968.2,374368.9346,1145796.944,10.366667,...,16423.15703,36983148,37044068,18,8,4,6,3.375177,33.881297,9.202895
4/28/2016,449.01,0.898219,66491.60486,0.904082,6.261128,178666000000.00,171720826.0,381601.8356,1207843.183,10.000000,...,18568.44670,37046272,37106607,24,12,7,5,5.008950,21.442079,26.105267


## Trying dropping column and checking accuracy

In [314]:
timeseries_test.drop(['n_transactions_excl_chains_longer_than_100','n_transactions_excl_popular','n_transactions_per_block','median_confirmation_time'],axis=1, inplace=True)
timeseries_test.drop(['mempool_count_stddev','mempool_growth_min'],axis=1, inplace=True)
timeseries_test.drop(['transactions_per_sec_min','transactions_per_sec_mean'],axis=1, inplace=True)
timeseries_test.drop(['mempool_size_stddev'],axis=1, inplace=True)
timeseries_test.drop(['utxo_count_min','utxo_count_max'],axis=1, inplace=True)
timeseries_test.drop(['mempool_count_max','n_transactions'],axis=1, inplace=True)



In [281]:
timeseries_test.drop(['n_transactions_excl_chains_longer_than_100','n_transactions_excl_popular','n_transactions_per_block','median_confirmation_time'],axis=1, inplace=True)

In [245]:
timeseries_test.head()

,close,avg_block_size,blocks_size,cost_per_transaction_percent,cost_per_transaction,difficulty,est_transaction_volume_USD,est_transaction_volume,hash_rate,median_confirmation_time,...,utxo_count_stddev,utxo_count_min,utxo_count_max,news_stories_total,news_stories_positive,news_stories_negative,news_stories_neutral,Negative_Tweets,Neutral_Tweets,Positive_Tweets
Date,,,,,,,,,,,,,,,,,,,,,
4/24/2016,458.55,0.728040,66022.67805,1.385967,8.360698,178678000000.00,110486293.2,240501.2912,1172443.384,8.783333,...,27228.75866,36724621,36825214,6,2,0,4,4.623191,8.705013,12.140564
4/25/2016,461.43,0.874485,66133.73765,0.888460,6.861613,178678000000.00,168025613.1,362233.4608,1128032.650,9.683333,...,17998.12914,36825828,36893715,25,7,8,10,4.402646,29.383710,28.087664
4/26/2016,466.09,0.813180,66256.52779,1.129962,7.675906,178678000000.00,170332016.5,363972.8546,1341204.175,7.750000,...,16768.83707,36895435,36980350,32,14,14,4,5.709683,13.106962,14.433717
4/27/2016,444.69,0.875343,66369.44708,0.873767,6.111057,178678000000.00,167458968.2,374368.9346,1145796.944,10.366667,...,16423.15703,36983148,37044068,18,8,4,6,1.791726,23.300896,7.735347
4/28/2016,449.01,0.898219,66491.60486,0.904082,6.261128,178666000000.00,171720826.0,381601.8356,1207843.183,10.000000,...,18568.44670,37046272,37106607,24,12,7,5,3.671006,15.149219,19.372779


In [315]:
X = timeseries_test.iloc[:, 1:]
y = timeseries_test.iloc[:, 0]

## CROSS VALIDATION APPROACH - less accuracy

In [180]:
from sklearn.model_selection import TimeSeriesSplit
scores = []
regression_model_cv = LinearRegression()
#cv = KFold(n_splits=10, random_state=42, shuffle=False)
tscv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tscv.split(X):

    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    regression_model_cv.fit(X_train, y_train)
    scores.append(regression_model_cv.score(X_test, y_test))

Train Index:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90] 

Test Index:  [ 91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176]
Train Index:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80 

In [181]:
y_predicted_cv = regression_model_cv.predict(X_test)
rmse_cv = mean_squared_error(y_test, y_predicted_cv)
rmse_cv

968546.2157401681

In [145]:
scores

[-0.32974232616126975,
 -0.049519492720137226,
 0.5938136794150712,
 -25.476112945492815,
 0.903487640292014,
 0.9377323309088349,
 0.8018815967600349,
 0.3539984567433092,
 -0.4895930665035091,
 -0.5053488557857828]

## Leave one out CV - worst

In [221]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

regression_model_cv_loo = LinearRegression()
for train_index, test_index in loo.split(X):
        #print("train:", train_index, "validation:", test_index)
        X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        regression_model_cv_loo.fit(X_train, y_train)
        scores.append(regression_model_cv_loo.score(X_test, y_test))

In [225]:
y_predicted_cv_loo = regression_model_cv_loo.predict(X_test)
rmse_cv_loo = mean_squared_error(y_test, y_predicted_cv_loo)
rmse_cv_loo

2365121.696245503

In [146]:
train_index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125
,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143
,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161
,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179
,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197
,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233
,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251
,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269
,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287
,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305
,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323
,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341
,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377
,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395
,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413
,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431
,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449
,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467
,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485
,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503
,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520],

test_index=[521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538
,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556
,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574
,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592
,593,594,595,596,597,598,599,600,601,602,603,604,605,606]

In [176]:
regression_model_cv_best = LinearRegression()
X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
regression_model_cv_best.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [177]:
y_predicted_2 = regression_model_cv_best.predict(X_test)

In [178]:
rmse_2 = mean_squared_error(y_test, y_predicted_2)

In [179]:
rmse_2

1216042.4731412642

In [149]:
print(regression_model_cv_best.score(X_test, y_test))

0.9377323309088349


# NON CV APPROACH  - better accuracy

In [316]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [8]:
# split into input and outputs
#train_X, train_y = train[:, 1:], train[:, 0]
#test_X, test_y = test[:, 1:], test[:, 0]

In [ ]:
# SCALING NOT REQUIRED IN L REGRESSION

#x_train_scaler = MinMaxScaler(feature_range=(0, 1))
#y_train_scaler = MinMaxScaler(feature_range=(0, 1))

#x_test_scaler = MinMaxScaler(feature_range=(0, 1))
#y_test_scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
#train_X = x_train_scaler.fit_transform(train_X)
#test_X = x_test_scaler.fit_transform(test_X)

In [317]:
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X_train, y_train)
# Predict
y_predicted = regression_model.predict(X_test)

# model evaluation
rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)


In [318]:
rmse
# for test size 0.2 - 119955.45035945765
# for test size 0.1 - 112537.56198911155
# for test size 0.01 - 42118.03027731108

98365.0438650796

In [233]:
X_test.shape

(96, 44)

## PREDICT

In [14]:
test_predict = pd.read_csv('../Data/Output/test_timeseries_merged.csv',encoding='latin1')
test_predict.set_index('Date', inplace=True)
actual_df = pd.read_csv('../Data/bitcoin_actual.csv',encoding='latin1')

In [15]:
actual_df.head()

,Price
0,4243.0
1,4159.7
2,3901.2
3,3981.9
4,3765.3


In [320]:
test_predict.drop(['n_transactions_excl_chains_longer_than_100','n_transactions_excl_popular','n_transactions_per_block','mempool_count_stddev','mempool_growth_min','median_confirmation_time','transactions_per_sec_min','transactions_per_sec_mean','mempool_size_stddev','utxo_count_min','utxo_count_max','mempool_count_max','n_transactions'],axis=1, inplace=True)

In [287]:
test_predict.drop(['n_transactions_excl_chains_longer_than_100','n_transactions_excl_popular','n_transactions_per_block','median_confirmation_time'],axis=1, inplace=True)

In [321]:
test_predict.head(5)

,avg_block_size,blocks_size,cost_per_transaction_percent,cost_per_transaction,difficulty,est_transaction_volume_USD,est_transaction_volume,hash_rate,miners_revenue,n_unique_addresses,...,transactions_per_sec_max,utxo_count_mean,utxo_count_stddev,news_stories_total,news_stories_positive,news_stories_negative,news_stories_neutral,Negative_Tweets,Neutral_Tweets,Positive_Tweets
Date,,,,,,,,,,,,,,,,,,,,,
12/1/2018,1.068438,192924.029808,0.923090,26.914157,6.653303e+12,7.051436e+08,171285.334800,41011449.14,6509108.18,420224.0,...,20.683333,58380899.25,12711.208037,915.0,297.0,508.0,110.0,50.952893,113.726201,130.891052
12/2/2018,1.005272,193038.630789,1.715932,27.105352,6.653303e+12,3.517405e+08,84399.905117,37704074.21,6035630.09,364378.0,...,24.466667,58397569.04,12797.050623,879.0,287.0,497.0,95.0,47.164804,150.869418,192.540884
12/3/2018,1.101028,193164.148009,0.455677,24.006523,6.194016e+12,1.267830e+09,319552.043684,31998005.40,5777217.67,426799.0,...,26.566667,58430813.58,9615.069463,1119.0,386.0,552.0,181.0,50.645421,221.967934,179.629582
12/4/2018,1.083460,193313.665436,0.279035,25.753689,5.646404e+12,2.493726e+09,629491.330519,38734427.59,6958363.40,492964.0,...,26.100000,58453246.33,9025.975571,1112.0,383.0,541.0,188.0,55.127732,148.565647,178.383022
12/5/2018,1.036768,193448.445287,0.442812,24.839415,5.646404e+12,1.438836e+09,372914.991753,36488953.53,6371334.81,438437.0,...,26.050000,58496345.79,9608.661188,1040.0,359.0,532.0,149.0,81.644125,166.521024,189.118929


In [322]:
test_predict['difficulty'] = test_predict['difficulty'].apply(lambda x: '{:.2f}'.format(x))
test_predict['est_transaction_volume_USD'] = test_predict['est_transaction_volume_USD'].apply(lambda x: '{:.2f}'.format(x))

In [323]:
X_test = test_predict.copy()

In [240]:
timeseries_test.columns

Index(['close', 'avg_block_size', 'blocks_size',
       'cost_per_transaction_percent', 'cost_per_transaction', 'difficulty',
       'est_transaction_volume_USD', 'est_transaction_volume', 'hash_rate',
       'miners_revenue', 'n_transactions', 'n_unique_addresses',
       'output_volume', 'total_bitcoins', 'trade_volume',
       'transaction_fees_USD', 'transaction_fees', 'mempool_count_mean',
       'mempool_count_stddev', 'mempool_count_min', 'mempool_count_max',
       'mempool_growth_mean', 'mempool_growth_stddev', 'mempool_growth_min',
       'mempool_growth_max', 'mempool_size_mean', 'mempool_size_stddev',
       'mempool_size_min', 'mempool_size_max', 'wallets_created',
       'transactions_per_sec_mean', 'transactions_per_sec_stddev',
       'transactions_per_sec_min', 'transactions_per_sec_max',
       'utxo_count_mean', 'utxo_count_stddev', 'utxo_count_min',
       'utxo_count_max', 'news_stories_total', 'news_stories_positive',
       'news_stories_negative', 'news_stories_

In [241]:
test_predict.columns

Index(['avg_block_size', 'blocks_size', 'cost_per_transaction_percent',
       'cost_per_transaction', 'difficulty', 'est_transaction_volume_USD',
       'est_transaction_volume', 'hash_rate', 'median_confirmation_time',
       'miners_revenue', 'n_transactions', 'n_unique_addresses',
       'output_volume', 'total_bitcoins', 'trade_volume',
       'transaction_fees_USD', 'transaction_fees', 'mempool_count_mean',
       'mempool_count_stddev', 'mempool_count_min', 'mempool_count_max',
       'mempool_growth_mean', 'mempool_growth_stddev', 'mempool_growth_min',
       'mempool_growth_max', 'mempool_size_mean', 'mempool_size_stddev',
       'mempool_size_min', 'mempool_size_max', 'wallets_created',
       'transactions_per_sec_mean', 'transactions_per_sec_stddev',
       'transactions_per_sec_min', 'transactions_per_sec_max',
       'utxo_count_mean', 'utxo_count_stddev', 'utxo_count_min',
       'utxo_count_max', 'news_stories_total', 'news_stories_positive',
       'news_stories_negati

In [324]:
y_predicted_test = regression_model.predict(X_test)
#y_predicted_test = regression_model_cv_best.predict(X_test)


In [21]:
actual = actual_df.values
actual = actual[:, 0]

### CALCULATE MAPE OF PREDICTION

In [22]:
#rmse_pred = mean_squared_error(actual, y_predicted_test)
def mape_calc(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true)) * 100



In [326]:
print(100 - mape_calc(actual,y_predicted_test))
#91.58090078218184 for 0.1 split
#91.6555509633192 with weights

91.6555509633192


In [327]:
df_pred_test = pd.DataFrame(y_predicted_test, index=y_predicted_test, columns=['close'])

In [328]:
df_pred_test.reset_index(inplace=True)

In [329]:
test_predict_wo_index = test_predict.reset_index()

In [330]:
test_predict_wo_index.rename(columns = {"Date":"date"}, inplace=True)

In [198]:
test_predict_wo_index.head()

,date,avg_block_size,blocks_size,cost_per_transaction_percent,cost_per_transaction,difficulty,est_transaction_volume_USD,est_transaction_volume,hash_rate,miners_revenue,...,transactions_per_sec_max,utxo_count_mean,utxo_count_stddev,news_stories_total,news_stories_positive,news_stories_negative,news_stories_neutral,Negative_Tweets,Neutral_Tweets,Positive_Tweets
0,12/1/2018,1.068438,192924.029808,0.923090,26.914157,6653303141405.00,705143612.38,171285.334800,41011449.14,6509108.18,...,20.683333,58380899.25,12711.208037,915.0,297.0,508.0,110.0,37.676332,80.636648,93.948681
1,12/2/2018,1.005272,193038.630789,1.715932,27.105352,6653303141405.00,351740543.24,84399.905117,37704074.21,6035630.09,...,24.466667,58397569.04,12797.050623,879.0,287.0,497.0,95.0,33.832619,113.716915,128.967312
2,12/3/2018,1.101028,193164.148009,0.455677,24.006523,6194015746025.25,1267830455.83,319552.043684,31998005.40,5777217.67,...,26.566667,58430813.58,9615.069463,1119.0,386.0,552.0,181.0,34.938430,155.527135,124.085115
3,12/4/2018,1.083460,193313.665436,0.279035,25.753689,5646403851534.00,2493725709.24,629491.330519,38734427.59,6958363.40,...,26.100000,58453246.33,9025.975571,1112.0,383.0,541.0,188.0,37.824774,103.030133,128.325976
4,12/5/2018,1.036768,193448.445287,0.442812,24.839415,5646403851534.00,1438836247.67,372914.991753,36488953.53,6371334.81,...,26.050000,58496345.79,9608.661188,1040.0,359.0,532.0,149.0,60.583100,125.615671,141.835649


In [199]:
df_pred_test.head()

,index,close
0,4995.153213,4995.153213
1,4960.941996,4960.941996
2,5133.098262,5133.098262
3,4180.858895,4180.858895
4,4038.788514,4038.788514


In [331]:
df_pred_test_concat = pd.concat((test_predict_wo_index['date'], df_pred_test['close'] ), axis=1)

In [332]:
df_pred_test_concat.to_csv('../Data/Output/df_pred_test_concat.csv',index=False)

# Selected Feature regression

In [129]:
selected_columns = ['difficulty','est_transaction_volume_USD','trade_volume','wallets_created','mempool_size_min','mempool_size_mean','mempool_size_max','hash_rate','miners_revenue','transaction_fees_USD','mempool_size_stddev','utxo_count_max','utxo_count_mean','utxo_count_min','output_volume','mempool_count_mean','mempool_count_min','est_transaction_volume','mempool_count_max','n_unique_addresses','utxo_count_stddev','total_bitcoins','mempool_growth_max','blocks_size','n_transactions','n_transactions_excl_popular','n_transactions_excl_chains_longer_than_100','mempool_count_stddev','news_stories_total','news_stories_negative','news_stories_positive','mempool_growth_stddev','transaction_fees','news_stories_neutral','mempool_growth_mean','n_transactions_per_block','mempool_growth_min','Neutral_Tweets','Positive_Tweets','cost_per_transaction','Negative_Tweets','transactions_per_sec_max','median_confirmation_time','cost_per_transaction_percent','transactions_per_sec_min','transactions_per_sec_mean']

In [138]:
#important - transactions_per_sec_mean

In [130]:
#timeseries_test_single = timeseries_test[['close','difficulty','est_transaction_volume_USD','trade_volume','wallets_created','mempool_size_min','mempool_size_mean','mempool_size_max','hash_rate','miners_revenue','transaction_fees_USD','mempool_size_stddev','utxo_count_max','utxo_count_mean','utxo_count_min','output_volume','mempool_count_mean','mempool_count_min','est_transaction_volume','mempool_count_max','n_unique_addresses','utxo_count_stddev','total_bitcoins','mempool_growth_max','blocks_size','n_transactions','n_transactions_excl_popular','n_transactions_excl_chains_longer_than_100','mempool_count_stddev','news_stories_total','news_stories_negative','news_stories_positive','mempool_growth_stddev','transaction_fees','news_stories_neutral','mempool_growth_mean','n_transactions_per_block','mempool_growth_min','Neutral_Tweets','Positive_Tweets','cost_per_transaction','Negative_Tweets']]
timeseries_test = pd.read_csv('../Data/Output/timeseries_merged.csv',encoding='latin1')
timeseries_test.set_index('Date', inplace=True)
timeseries_test['difficulty'] = timeseries_test['difficulty'].apply(lambda x: '{:.2f}'.format(x))
timeseries_test_single = timeseries_test.copy()

In [131]:
#X = timeseries_test_single.iloc[:, 1:]
X = timeseries_test_single[selected_columns]
#y = timeseries_test_single.iloc[:, 0]
y = timeseries_test_single['close']

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


In [133]:
regression_model_single = LinearRegression()
# Fit the data(train the model)
regression_model_single.fit(X_train, y_train)
# Predict
y_predicted_single = regression_model_single.predict(X_test)

# model evaluation
rmse = mean_squared_error(y_test, y_predicted_single)
r2 = r2_score(y_test, y_predicted_single)


In [134]:
rmse

99975.55070367844

### PREDICT ON TEST DATA

In [135]:
#test_predict_single = test_predict[['difficulty','est_transaction_volume_USD','trade_volume','wallets_created','mempool_size_min','mempool_size_mean','mempool_size_max','hash_rate','miners_revenue','transaction_fees_USD','mempool_size_stddev','utxo_count_max','utxo_count_mean','utxo_count_min','output_volume','mempool_count_mean','mempool_count_min','est_transaction_volume','mempool_count_max','n_unique_addresses','utxo_count_stddev','total_bitcoins','mempool_growth_max','blocks_size','n_transactions','n_transactions_excl_popular','n_transactions_excl_chains_longer_than_100','mempool_count_stddev','news_stories_total','news_stories_negative','news_stories_positive','mempool_growth_stddev','transaction_fees','news_stories_neutral','mempool_growth_mean','n_transactions_per_block','mempool_growth_min','Neutral_Tweets','Positive_Tweets','cost_per_transaction','Negative_Tweets']]
test_predict = pd.read_csv('../Data/Output/test_timeseries_merged.csv',encoding='latin1')
test_predict.set_index('Date', inplace=True)

test_predict_single = test_predict[selected_columns]

In [136]:
X_test_single = test_predict_single.copy()
y_predicted_test_single = regression_model_single.predict(X_test_single)
df_pred_test_single = pd.DataFrame(y_predicted_test_single, index=y_predicted_test_single, columns=['Close'])
df_pred_test_single.reset_index(inplace=True)


In [137]:
print(100 - mape_calc(actual,y_predicted_test_single))

90.68769797518908


In [20]:
X_test_single.shape

(113, 41)

In [ ]:
df_pred_test_concat_single = pd.concat((test_predict_wo_index['Date'], df_pred_test_single['Close'] ), axis=1)
df_pred_test_concat_single.to_csv('../Data/Output/df_pred_test_concat_single.csv',index=False)